In [1]:
import pandas as pd
import pickle

In [2]:
df = pd.read_csv("../train_scaled.csv", index_col=0)
df
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [3]:
X

,/JS,/AA,/OpenAction,/AcroForm,/RichMedia,/Launch,/EmbeddedFile,/XFA,/URI,/F,...,chunk,base64,replace,fromCharCode,getAnnots,ip,xref_size,trailer_size,file_size,nof_stream
file_name,,,,,,,,,,,,,,,,,,,,,
file_0001,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.181818,0.190909,0.129630,0.349614,0.054348
file_0012,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.109091,0.092593,0.118348,0.054348
file_0018,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.272727,0.272727,0.259259,0.087789,0.304348
file_0022,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.181818,0.190909,0.129630,0.384506,0.054348
file_0023,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.109091,0.092593,0.144489,0.054348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
file_2891,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.272727,0.000000,0.000000,0.106057,0.260870
file_2899,0.666667,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.081818,0.055556,0.052075,0.021739
file_2902,1.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.454545,0.209091,0.141975,0.075135,0.206522


In [4]:
y

file_name
file_0001    0
file_0012    0
file_0018    0
file_0022    0
file_0023    0
            ..
file_2891    1
file_2899    1
file_2902    1
file_2932    1
file_2955    1
Name: class, Length: 367, dtype: int64

In [5]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)

In [6]:
def train(model, X, y):
	# 모델 학습
	model.fit(X, y)

In [7]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
def get_clf_eval(y, y_pred, vervose=True):
	confusion = confusion_matrix(y, y_pred)
	accuracy = accuracy_score(y, y_pred)
	precision = precision_score(y, y_pred)
	recall = recall_score(y, y_pred)
	F1 = f1_score(y, y_pred)
	AUC = roc_auc_score(y, y_pred)
	if vervose:
		print('오차행렬:\n', confusion)
		print('\n정확도: {:.4f}'.format(accuracy))
		print('정밀도: {:.4f}'.format(precision))
		print('재현율: {:.4f}'.format(recall))
		print('F1: {:.4f}'.format(F1))
		print('AUC: {:.4f}'.format(AUC))
	return confusion, accuracy, precision, recall, F1, AUC


# hold out

In [8]:
# scikit-learn 패키지의 train_test_split()을 불러옵니다.
from sklearn.model_selection import train_test_split
# 학습 데이터와 학습용과 검증용으로 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y,
													test_size=0.2,
													random_state=42)
print(f"Training Shape: {X.shape[0]}, Test Shape: {X_test.shape[0]}")
train(model, X_train, y_train)
y_pred = model.predict(X_test)
confusion, accuracy, precision, recall, F1, AUC = get_clf_eval(y_test, y_pred)

Training Shape: 367, Test Shape: 74
오차행렬:
 [[52  1]
 [ 1 20]]

정확도: 0.9730
정밀도: 0.9524
재현율: 0.9524
F1: 0.9524
AUC: 0.9668


# cross validation

In [9]:
import os
os.getcwd()


'D:\\remote_workpace\\CSRC_training\\AI_detecting\\PDF_DATA\\AI'

In [10]:
from sklearn.model_selection import KFold
knum = 10
kfold = KFold(n_splits=knum, shuffle=True, random_state=42)
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)
max_acc, max_f1, max_auc = 0, 0, 0
for train_index, test_index in kfold.split(X):
	X_train, X_test = X.iloc[train_index], X.iloc[test_index]
	y_train, y_test = y.iloc[train_index], y.iloc[test_index]
	# 학습 및 예측
	train(model, X_train, y_train)
	y_pred = model.predict(X_test)
	_, accuracy, _, _, F1, AUC = get_clf_eval(y_test, y_pred)
	if max_acc < accuracy:
		max_acc = accuracy
		with open('../models/max_acc.pickle','wb') as fw:
			pickle.dump(model, fw)
	if max_f1 < F1:
		max_f1 = F1
		with open('../models/max_f1.pickle','wb') as fw:
			pickle.dump(model, fw)
	if max_auc < AUC:
		max_auc = AUC
		with open('../models/max_auc.pickle','wb') as fw:
			pickle.dump(model, fw)


오차행렬:
 [[28  0]
 [ 1  8]]

정확도: 0.9730
정밀도: 1.0000
재현율: 0.8889
F1: 0.9412
AUC: 0.9444
오차행렬:
 [[24  1]
 [ 0 12]]

정확도: 0.9730
정밀도: 0.9231
재현율: 1.0000
F1: 0.9600
AUC: 0.9800
오차행렬:
 [[23  1]
 [ 0 13]]

정확도: 0.9730
정밀도: 0.9286
재현율: 1.0000
F1: 0.9630
AUC: 0.9792
오차행렬:
 [[32  0]
 [ 0  5]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[25  1]
 [ 0 11]]

정확도: 0.9730
정밀도: 0.9167
재현율: 1.0000
F1: 0.9565
AUC: 0.9808
오차행렬:
 [[23  1]
 [ 0 13]]

정확도: 0.9730
정밀도: 0.9286
재현율: 1.0000
F1: 0.9630
AUC: 0.9792
오차행렬:
 [[29  1]
 [ 1  6]]

정확도: 0.9459
정밀도: 0.8571
재현율: 0.8571
F1: 0.8571
AUC: 0.9119
오차행렬:
 [[26  0]
 [ 2  8]]

정확도: 0.9444
정밀도: 1.0000
재현율: 0.8000
F1: 0.8889
AUC: 0.9000
오차행렬:
 [[25  0]
 [ 1 10]]

정확도: 0.9722
정밀도: 1.0000
재현율: 0.9091
F1: 0.9524
AUC: 0.9545
오차행렬:
 [[26  0]
 [ 0 10]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000


In [11]:
from sklearn.model_selection import StratifiedKFold
knum = 10
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)
skf = StratifiedKFold(n_splits=knum)
max_acc, max_f1, max_auc = 0, 0, 0
acc_list, recall_list, precision_list, f1_list = [], [], [], []
for train_index, test_index in skf.split(X, y):
	X_train, X_test = X.iloc[train_index], X.iloc[test_index]
	y_train, y_test = y.iloc[train_index], y.iloc[test_index]
	train(model, X_train, y_train)
	y_pred = model.predict(X_test)
	_, accuracy, precision, recall, F1, AUC = get_clf_eval(y_test, y_pred)
	acc_list.append(accuracy)
	recall_list.append(recall)
	precision_list.append(precision)
	f1_list.append(F1)
	if max_acc < accuracy:
		max_acc = accuracy
		with open('../models/max_acc.pickle','wb') as fw:
			pickle.dump(model, fw)
	if max_f1 < F1:
		max_f1 = F1
		with open('../models/max_f1.pickle','wb') as fw:
			pickle.dump(model, fw)
	if max_auc < AUC:
		max_auc = AUC
		with open('../models/max_auc.pickle','wb') as fw:
			pickle.dump(model, fw)


오차행렬:
 [[27  0]
 [ 1  9]]

정확도: 0.9730
정밀도: 1.0000
재현율: 0.9000
F1: 0.9474
AUC: 0.9500
오차행렬:
 [[27  0]
 [ 0 10]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[26  1]
 [ 0 10]]

정확도: 0.9730
정밀도: 0.9091
재현율: 1.0000
F1: 0.9524
AUC: 0.9815
오차행렬:
 [[27  0]
 [ 0 10]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[26  1]
 [ 0 10]]

정확도: 0.9730
정밀도: 0.9091
재현율: 1.0000
F1: 0.9524
AUC: 0.9815
오차행렬:
 [[26  1]
 [ 0 10]]

정확도: 0.9730
정밀도: 0.9091
재현율: 1.0000
F1: 0.9524
AUC: 0.9815
오차행렬:
 [[26  0]
 [ 2  9]]

정확도: 0.9459
정밀도: 1.0000
재현율: 0.8182
F1: 0.9000
AUC: 0.9091
오차행렬:
 [[24  2]
 [ 0 10]]

정확도: 0.9444
정밀도: 0.8333
재현율: 1.0000
F1: 0.9091
AUC: 0.9615
오차행렬:
 [[26  0]
 [ 0 10]]

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1: 1.0000
AUC: 1.0000
오차행렬:
 [[26  0]
 [ 1  9]]

정확도: 0.9722
정밀도: 1.0000
재현율: 0.9000
F1: 0.9474
AUC: 0.9500


In [12]:
print(f"----------Max----------\nAccuracy: {max_acc}, F1: {max_f1}, AUC: {max_auc}")
print(f"----------Average----------\nAccuracy: {sum(acc_list)/knum}, Recall: {sum(recall_list)/knum}, Precision: {sum(precision_list)/knum}, F1: {sum(f1_list)/knum}")

----------Max----------
Accuracy: 1.0, F1: 1.0, AUC: 1.0
----------Average----------
Accuracy: 0.9754504504504503, Recall: 0.9618181818181819, Precision: 0.956060606060606, F1: 0.956097060833903
